# Direct Model
## Setup

In [8]:
from data_utils import *
from train import * 

import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.models as models

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

use_cuda = torch.cuda.is_available()
print("GPU Available:", use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
dtype = torch.float32

print('using device:', device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GPU Available: False
using device: cpu


## Load Dataset

In [9]:
data = load_bolete_data()
print()
for k in data.keys():
    print(k, np.shape(data[k]))

ItemsViewHDF5(<HDF5 file "bolete.h5" (mode r)>)

bolete-characteristics (38, 1868)
bolete-edibility (5, 1868)
bolete-images (3, 512, 512, 1868)
bolete-labels (1868,)


## Split into train and test

In [10]:
X_train, X_test, Y_train, Y_test = get_train_and_test(data, "bolete-labels")
N, H, W, C = X_train.shape
M = np.size(np.unique(Y_train))

Y_train = Y_train.astype(np.long)
Y_test = Y_test.astype(np.long)

X_train = np.transpose(X_train,[0,3,1,2])
print(X_train.shape)

(1307, 3, 512, 512)


## Define models

In [11]:
def simple_model():
    # copy final model from ass igment 2
    def flatten(x):
        """Flattens to [N, -1] where -1 is whatever it needs to be"""
        N = x.shape[0] # read in N, C, H, W
        return x.view(N, -1)  
        # "flatten" the C * H * W values into a single vector per image
    class Flatten(nn.Module):
        def forward(self, x):
            return flatten(x)

    channel_1 = 32
    channel_2 = 24
    channel_3 = 16
    hidden_dim = 150
    torch.manual_seed(0)
    learning_rate = 3e-3 # 1e-2

    model = nn.Sequential(
        nn.Conv2d(C, channel_1, kernel_size=5, padding=2),
        # nn.GroupNorm(4,channel_1),
        nn.BatchNorm2d(channel_1),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1),
        # nn.GroupNorm(4,channel_2),
        nn.BatchNorm2d(channel_2),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Conv2d(channel_2, channel_3, kernel_size=3, padding=1),
        # nn.GroupNorm(4,channel_3),
        nn.BatchNorm2d(channel_3),
        nn.ReLU(),
        nn.Dropout(0.1),
        Flatten(),
        nn.Linear(channel_3 * H * W, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, M)
    )
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                        momentum=0.8, nesterov=True)
    return model, optimizer


## Define the loss function

In [15]:
loss_fn = nn.CrossEntropyLoss()

def pred_fn(scores):
    m = nn.Softmax(dim=1)
    return m(scores)

## Cross Validate on training data

In [13]:
model, optimizer = simple_model()
transform = T.Compose([T.ToTensor()])

cross_val(
    X_train=X_train,
    Y_train=Y_train,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    pred_fn=pred_fn,
    batch_size=64,
    num_epochs=1,
    show_every=5,
    device=device,
    dtype=dtype,
    transform=transform,
)

tensor([[False,  True,  True,  ...,  True, False,  True],
        [False,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False,  True],
        ...,
        [ True,  True,  True,  ..., False, False,  True],
        [ True, False,  True,  ..., False, False, False],
        [ True,  True,  True,  ...,  True, False,  True]])


RuntimeError: The size of tensor a (172) must match the size of tensor b (64) at non-singleton dimension 1

### Check status of GPU Memory for Debugging Purposes

In [14]:
import torch
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

<class 'torch.nn.parameter.Parameter'> torch.Size([32, 3, 5, 5])
<class 'torch.nn.parameter.Parameter'> torch.Size([32])
<class 'torch.nn.parameter.Parameter'> torch.Size([32])
<class 'torch.nn.parameter.Parameter'> torch.Size([32])
<class 'torch.nn.parameter.Parameter'> torch.Size([24, 32, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([24])
<class 'torch.nn.parameter.Parameter'> torch.Size([24])
<class 'torch.nn.parameter.Parameter'> torch.Size([24])
<class 'torch.nn.parameter.Parameter'> torch.Size([16, 24, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([16])
<class 'torch.nn.parameter.Parameter'> torch.Size([16])
<class 'torch.nn.parameter.Parameter'> torch.Size([16])
<class 'torch.nn.parameter.Parameter'> torch.Size([150, 4194304])
<class 'torch.nn.parameter.Parameter'> torch.Size([150])
<class 'torch.nn.parameter.Parameter'> torch.Size([172, 150])
<class 'torch.nn.parameter.Parameter'> torch.Size([172])
<class 'torch.Tensor'> torch.Size([32, 3, 5, 5])
<class '